# Метод Ньютона

In [1]:
from sympy import diff, hessian, symbols, Abs, Max
from numpy import matrix,squeeze,asarray, set_printoptions
from numpy.linalg import inv

set_printoptions(precision=3, suppress=True)

global epsilon
epsilon = 1E-5

def fValue(f, x0, syms):
    length = range(0, len(x0))
    return f.subs([(syms[i], x0[i]) for i in length]) 

def gradValue(gradF, x0, syms):
    length = range(0, len(x0))
    gradAsList = [gradF[j].subs([(syms[i], x0[i]) for i in length]) for j in length]
    return matrix(gradAsList, 'float')

def hessianValue(hessF, x0, syms):
    lQ = range(0, len(hessF))
    lX = range(0, len(x0))
    a = matrix([hessF[j].subs([(syms[i], x0[i]) for i in lX]) for j in lQ], 'float')
    size = int(len(hessF) ** 0.5)
    return matrix(a.reshape(size, size))

In [2]:
class NewtonOptimizer:
    def __init__(self, f, x0, syms):
        """
        f     = function representation (sympy)
        x0    = Initial guess of the optimizer
        syms  = variables on which f depend
        """
        self._name = "Newton's unrestrained optimiztion"
        self._f = f
        self._x = x0
        self._syms = syms

    def __repr__(self):
        """
        Description : Prints the minimizer of the function on the screen
        
        Inputs: 
               f  = symbolic representation of the function to be minimized
               x0 = Initial guess of the minimizer
        """
        return_string  = "\n" + "=" * 20 + self._name + "=" * 20
        return_string += "\n" + "Function to be optimized:" + f"{self._f}" + "\n" * 2
        return_string += f"Optimal parameter is: {self._x}" + "\n" * 2
        return_string += f"The optimal value of f is: %.3f" % fValue(self._f, self._x, self._syms)
        return_string += "\n" * 2 + "=" * (40 + len(self._name))
        return return_string

    def optimize(self):
        gradF = [self._f.diff(x) for x in self._syms]
        hessianF = hessian(self._f, self._syms)
        while (True):
            g0 = gradValue(gradF, self._x, self._syms)
            if (all(abs(i) < epsilon for i in squeeze(asarray(g0,'float')))): 
                return self._x, fValue(self._f, self._x, self._syms)
            F0 = hessianValue(hessianF, self._x, self._syms)
            self._x -= squeeze(asarray((inv(F0) * g0.transpose()).transpose()))

In [3]:
x, y = symbols('x y')
syms = [x, y]

In [4]:
f_rosenbrock = (1 + x)**2 + 100*(y - x**2)**2

In [5]:
optimizer = NewtonOptimizer(f_rosenbrock, [0, 0], syms)
x_opt, f_opt = optimizer.optimize()
x_opt

array([-1.,  1.])

In [6]:
f_opt

4.93038065763132e-30

In [7]:
optimizer


====================Newton's unrestrained optimiztion====================
Function to be optimized:(x + 1)**2 + 100*(-x**2 + y)**2

Optimal parameter is: [-1.  1.]

The optimal value of f is: 0.000


# Метод барьерных функций

**Пусть имеется следующая задача:**

Минимизировать $f(x)$ при ограничениях $g_i(x)\ge0,h_i(x)=0$ , где функции $f,g_i,h_i$.

Начальный этап Выбрать $\epsilon>0$ Выбрать начальную точку $x^1$, параметр штрафа $r_1$ и число $\beta>1$. Положить $k=1$ и перейти к основному этапу.

**Основной этап. k-я итерация.**

**Первый шаг.** При начальной точке $x_k$ и параметре штрафа $r_k$ решить следующую задачу:

минимизировать

$f(x)+r_k\alpha(x)=f(x)+r_k\left[\sum_{i=1}^{m}(max\{0,-g_i(x)\})^p+ \sum_{i=m+1}^{l}|h_i(x)|^p\right]$ , где
$p>0$ , $p$ - целое.

Положить $x_{k+1}$ равным оптимальному решению задачи и перейти ко второму шагу.

**Второй шаг**

Если $r_k\alpha(x_{k+1})<\epsilon$, то остановиться.

В противном случае положить $r_{k+1}=\beta r_k$. Заменить $k$ на $k+1$ и перейти к первому шагу.

In [8]:
class BarrierOptimizer:
    def __init__(self, f, gs, hs, x0, syms, eps = 0.01, r0 = 1., beta = 1.1, p = 2):
        self._name = "Barrier functions constrained optimization"
        self._eps = eps
        self._penalty = r0
        self._beta = beta
        self._p = p

        self._f = f
        self._gs = gs
        self._hs = hs
        self._x = x0
        self._syms = syms

    def __repr__(self):
        return_string  = "\n" + "=" * 20 + self._name + "=" * 20

        return_string += "\n" + "Function to be optimized: " + f"{self._f}" + "\n" * 2
        return_string += "Constraints:" + "\n"
        for g in self._gs:
            return_string += f"{g}>0" + "\n"
        for g in self._hs:
            return_string += f"{h}=0" + "\n"
        return_string += "\n" * 2 + f"The optimal point: {self._x}" + "\n"
        return_string += f"The optimal value: {fValue(self._f, self._x, self._syms):.3f}"
        return_string += "\n" * 2 + "=" * (40 + len(self._name)) + "\n"
        
        return return_string
        

    def optimize(self):
        flag = False
        i = 0

        return_string  = "\n" + "=" * 20 + self._name + "=" * 20

        return_string += "\n" + "Function to be optimized: " + f"{self._f}" + "\n" 
        return_string += f"The initial value: {self._x}" + "\n" * 2
        return_string += "Constraints:" + "\n"
        for g in self._gs:
            return_string += f"{g}>0" + "\n"
        for g in self._hs:
            return_string += f"{h}=0" + "\n"

        return_string += "\n" * 2 + "Training:"
        print(return_string)

        while flag is False:
            alpha = 0

            for g in self._gs:
                alpha += Max(0, -g) ** self._p

            for h in self._hs:
                alpha += Abs(h) ** self._p

            f = self._f + self._penalty * alpha

            optimizer = NewtonOptimizer(f, self._x, self._syms)
            self._x, f_opt = optimizer.optimize()

            if self._penalty * fValue(alpha, self._x, self._syms) < self._eps:
                flag = True
            else:
                self._penalty *= self._beta

            print(f"\tIteration {i}: x = {self._x} f = {f_opt:.3f}")
            i += 1

        print("\n" * 2 + "=" * (40 + len(self._name)))

        return self._x, f_opt

In [9]:
x_1, x_2, x_3, x_4 = symbols('x_1 x_2 x_3 x_4')
syms = [x_1, x_2, x_3, x_4]

f = x_1 ** 2 + x_2 ** 2 + 2 * x_3 ** 2 + x_4 ** 2 - 5 * x_1 - 5 * x_2 - 21 * x_3 + 7 * x_4
g1 = 8 - x_1 ** 2 - x_2 ** 2 - x_3 ** 2 - x_4 ** 2 - x_1 + x_2 - x_3 + x_4
g2 = 10 - x_1 ** 2 - 2 * x_2 ** 2 - x_3 ** 2 - 2 * x_4 ** 2 + x_1 - x_4
g3 = 5 - 2 * x_1 ** 2 - x_2 ** 2 - x_3 ** 2 - 2 * x_1 + x_2 + x_4
g = [g1, g2, g3]

bo = BarrierOptimizer(f, g, [], (0, 0, 0, 0), syms)

In [10]:
print(*syms)

x_1 x_2 x_3 x_4


In [11]:
x_opt, y_opt = bo.optimize()


====================Barrier functions constrained optimization====================
Function to be optimized: x_1**2 - 5*x_1 + x_2**2 - 5*x_2 + 2*x_3**2 - 21*x_3 + x_4**2 + 7*x_4
The initial value: (0, 0, 0, 0)

Constraints:
-x_1**2 - x_1 - x_2**2 + x_2 - x_3**2 - x_3 - x_4**2 + x_4 + 8>0
-x_1**2 + x_1 - 2*x_2**2 - x_3**2 - 2*x_4**2 - x_4 + 10>0
-2*x_1**2 - 2*x_1 - x_2**2 + x_2 - x_3**2 + x_4 + 5>0


Training:
	Iteration 0: x = [ 0.094  1.047  2.118 -0.959] f = -45.007
	Iteration 1: x = [ 0.089  1.044  2.11  -0.956] f = -44.926
	Iteration 2: x = [ 0.084  1.041  2.102 -0.954] f = -44.851
	Iteration 3: x = [ 0.079  1.038  2.095 -0.953] f = -44.782
	Iteration 4: x = [ 0.074  1.036  2.088 -0.952] f = -44.718
	Iteration 5: x = [ 0.07   1.034  2.082 -0.952] f = -44.659
	Iteration 6: x = [ 0.066  1.031  2.076 -0.952] f = -44.605
	Iteration 7: x = [ 0.062  1.029  2.07  -0.952] f = -44.555
	Iteration 8: x = [ 0.058  1.027  2.065 -0.953] f = -44.509
	Iteration 9: x = [ 0.054  1.026  2.061 -0.954

In [12]:
bo


====================Barrier functions constrained optimization====================
Function to be optimized: x_1**2 - 5*x_1 + x_2**2 - 5*x_2 + 2*x_3**2 - 21*x_3 + x_4**2 + 7*x_4

Constraints:
-x_1**2 - x_1 - x_2**2 + x_2 - x_3**2 - x_3 - x_4**2 + x_4 + 8>0
-x_1**2 + x_1 - 2*x_2**2 - x_3**2 - 2*x_4**2 - x_4 + 10>0
-2*x_1**2 - 2*x_1 - x_2**2 + x_2 - x_3**2 + x_4 + 5>0


The optimal point: [ 0.001  1.001  2.002 -0.998]
The optimal value: -44.019
